In [1]:
import pyspark    # 測試安裝成功
import pymongo
import pymongo_spark
from pymongo import MongoClient
pymongo_spark.activate()
client = MongoClient('10.120.28.20', 27017)
database = client["test"]
collection = database["news"]

In [2]:
#conf = {master:"localhost",appName:"test"}  # 測試運作成功
#sc = pyspark.SparkContext(conf = conf)
sc = pyspark.SparkContext(master="local[*]", appName="test")
rdd = sc.mongoPairRDD("mongodb://10.120.28.20:27017/test.news")
#resp_list
print type(rdd)



<class 'pyspark.rdd.RDD'>


In [5]:
import site  # 查看python import的路徑
site.getsitepackages()

['/usr/local/python/lib/python2.7/site-packages',
 '/usr/local/python/lib/site-python']

In [69]:
import pymongo_spark  # 以下為LDA作機器學習的資料前處理流程
import jieba
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# 載入字典給jieba
jieba.set_dictionary('/home/cloudera/Desktop/text_mining/dict.txt')            
jieba.load_userdict("/home/cloudera/Desktop/text_mining/dict_keyw_new.txt")
jieba.load_userdict("/home/cloudera/Desktop/text_mining/dict_cbdic.txt")

# 啟動Pymongo對pyspark的介面
pymongo_spark.activate() 

# 設定sc,載入mongodb的資料成一個RDD
#sc = pyspark.SparkContext(master="local[*]",appName="LDAtest")  # sparkcontext
mongo_rdd = sc.mongoRDD('mongodb://10.120.28.20:27017/test.LDAsample') # loader

Building prefix dict from /home/cloudera/Desktop/text_mining/dict.txt ...
Loading model from cache /tmp/jieba.u77727ee2cf0db5a92286247fca7f8515.cache
Loading model cost 0.536 seconds.
Prefix dict has been built succesfully.


In [72]:
from pyspark.mllib.linalg import Vectors
from operator import add
import re

content = mongo_rdd.map(lambda x: x['content']) # 撈出content欄位的資料組成RDD
print content.count() # 有268筆新聞文章資料

RDD_list=[] # 將切詞過的資料放到seg_RDD,再個別裝入list
for ele in content.collect():
    seg_list = jieba.cut(ele) 
    seg_RDD = sc.parallelize(seg_list)
    RDD_list.append(seg_RDD)
print RDD_list[0].first()

268
跨年


In [73]:
def sefilter(se):
    se = re.sub(
        "[\s+\.\!\/_,$%^*「」\:\-(+\"\']+|[的1234567890+——！，。？、~@#￥%……&*（）]+"\
                     .decode("utf8"), "".decode("utf8"),se).strip()
    # 刪去數字,符號與空白,若se剩下空字串傳回False
    # 偷懶把停詞"的"加在裡面...    
    if se =="":
        return False
    else:
        return True

seRDD_list=[]

for RDD in RDD_list:
    seRDD = RDD.filter(sefilter) 
    # filter(function()),return True的放進新的RDD,return False的不放進去
    seRDD_list.append(seRDD)

seRDD_list[0].collect()

[u'\u8de8\u5e74',
 u'\u5c08\u520a',
 u'\u3010',
 u'\u9023\u7dda',
 u'\u5831\u5c0e',
 u'\u3011',
 u'\u65b0\u5e74\u5feb\u6a02',
 u'\u5317',
 u'\u4e2d',
 u'\u5357\u4eca',
 u'\u51cc\u6668',
 u'\u96f6\u6642',
 u'\u540c\u6b65',
 u'\u4ee5',
 u'\u7d62\u721b',
 u'\u7159\u82b1',
 u'\u8fce\u63a5',
 u'\u5e74',
 u'\u53f0\u5317101',
 u'\u5927\u6a13',
 u'\u65bd\u653e',
 u'\u5176',
 u'\u53f2\u4e0a',
 u'\u6700\u9577',
 u'\u79d2',
 u'\u865f\u7a31',
 u'\u6b77\u4f86',
 u'\u5716\u6848',
 u'\u6700\u591a\u7684',
 u'\u842c\u767c',
 u'\u7159\u706b',
 u'\uff1b',
 u'\u96f2\u6797',
 u'\u528d\u6e56\u5c71\u4e16\u754c',
 u'\u4ee5',
 u'\u6469\u5929\u8f2a',
 u'\u9ad8\u7a7a',
 u'\u540a\u8eca',
 u'\u52a0\u8a2d',
 u'\u7d44',
 u'\u98a8\u706b\u8f2a',
 u'\u7d44\u6210',
 u'\u4e09\u74b0',
 u'\u7159\u706b',
 u'\u71e6\u721b',
 u'\u65bd\u653e',
 u'\uff1b',
 u'\u9ad8\u96c4',
 u'\u7fa9\u5927\u4e16\u754c',
 u'\u5247\u4ee5',
 u'\u5168\u53f0',
 u'\u6700\u9577',
 u'\u79d2',
 u'\u842c\u767c',
 u'\u7159\u706b',
 u'\u795d',
 u'\u570b\u4e

In [87]:
meta_list = []
for RDD in seRDD_list:
    metaRDD = RDD.map(lambda x:(x[0:],1)) # RDD轉型成(關鍵字,1)的格式
    meta_list.append(metaRDD)
#print len(meta_list)
print meta_list[0].first()

wordCount_list = []
wc_collect = []
for RDD in meta_list:   # 用reduce算次數,傳回結果是RDD,計算較快
    wcRDD = RDD.reduceByKey(lambda x,y: x+y).\
    filter(lambda i:i[1]>3)   # 過濾掉次數3以下的詞
    # reduceByKey()有suffle,排序會變動
    wordCount_list.append(wcRDD)
    wc_collect.append(dict(wcRDD.collect()))

print len(wc_collect[0])


(u'\u8de8\u5e74', 1)


In [89]:
key_set = set([])   # 利用python set的特性快速整理出key word清單
for ele in wc_collect:
    key_set = set.union(key_set, set(ele.keys())) # 做出每篇文章的key word聯集
    
print key_set 

set([u'\u571f\u9280', u'\u8a31\u5973', u'\u767c\u5c55', u'\u7cfb\u5217', u'\u7624', u'tw', u'\u576a\u6797', u'\u6848\u4ef6', u'\u694a\u5973', u'to', u'\u9802\u65b0', u'\u52f8\u52df', u'\u9ec3\u6625\u660e', u'\u7b2c\u4e8c\u8258', u'CONVERSE', u'\u5efa\u9020', u'\u548c\u725b', u'\u8868\u9762', u'BOOST', u'\u535a\u7269\u9928', u'\u638c\u5fc3', u'\u51fa\u570b', u'\u6842\u862d', u'\u7f6a', u'\u9928\u5916', u'\u6b72', u'\u5e74', u'\u52c7\u58eb', u'\u6eff', u'\u5546\u52d9', u'\u300b', u'\u7626\u8089\u7cbe', u'\u6295\u8cc7\u4eba', u'\u809b\u9580', u'\uff15', u'\u661f\u4f0a', u'\u5916\u5957', u'\u59d0\u59d0', u'\u816b\u7624', u'\u512a\u60e0', u'\u81ea\u5df1\u7684', u'\u5361\u724c', u'\u662f', u'\u4f4e\u9ede', u'\u53f0\u5f69', u'\u8cc7\u683c\u8cfd', u'\u96fb\u90f5', u'\u9ad8\u82d1\u5de5\u5546', u'\u8a02\u4f4d', u'\u5167\u5bb9', u'\u25ce', u'\u9069\u7528', u'\u7f8e\u570b', u'\u52a0\u5f37', u'\u6e96\u5099', u'\u821e\u8005', u'\u4e5f', u'\u83ef\u822a', u'\u624b\u81c2', u'\u8eca\u6b3e', u'\u590f\u53

In [91]:
# 從key word清單拿詞出來跟每篇文章切出來的詞作比對
# 若文章沒有該詞則新增(詞,0)的元素, 稍後用此資料建立稀疏矩陣

key_list = list(key_set)
for key in key_list:
    for ele in wc_collect:
        if key not in ele.keys():
            ele.__setitem__(key,0)
            
print len(wc_collect[0])
print wc_collect[0]

1061
1061
{u'\u6027\u4fb5': 0, u'\u7576\u4ee3': 0, u'\u85e5\u7269': 0, u'\u5287': 0, u'\u53bb\u5e74': 0, u'\u9ec3\u6625\u660e': 0, u'\u7f8e\u98df': 0, u'\u5546\u6a5f': 0, u'\u5973\u7db2\u53cb': 0, u'\u5916': 0, u'\uff1a': 0, u'\u512a\u60e0': 0, u'\u90ed\u4ed5\u714c': 0, u'\u7a0b\u5ea6\u8f03\u9ad8': 0, u'\u571f\u9280': 0, u'\u8a31\u5973': 0, u'\u767c\u5c55': 0, u'\u7cfb\u5217': 0, u'\u83ef\u822a': 0, u'\u6240\u4ee5': 0, u'tw': 0, u'\u9078\u5340': 0, u'\u576a\u6797': 0, u'\u6848\u4ef6': 0, u'\u694a\u5973': 0, u'to': 0, u'\u98ef\u5e97': 0, u'\u9802\u65b0': 0, u'\u52f8\u52df': 0, u'\u751f\u8096': 0, u'\u96fb\u5b50\u652f\u4ed8': 0, u'\u696d\u7e3e': 0, u'\u670d\u52d9': 0, u'\u5e0c\u671b': 0, u'\u5404\u5730': 0, u'\u653e': 0, u'\u7b2c\u4e8c\u8258': 0, u'\u5343\u74bd': 0, u'\u5206\u9418': 0, u'CONVERSE': 0, u'\u5efa\u9020': 0, u'\u8a2d\u8a08': 0, u'\u806f\u76df': 0, u'\u8b66\u54e1': 0, u'\u5e73\u53cd': 0, u'\u8868\u9762': 0, u'\u4ee5': 0, u'\u5f90\u8389\u73b2': 0, u'\u7a7a\u96e3': 0, u'BOOST':

In [98]:
# 將key word清單存檔
s = ""
for keyw in wc_collect[0].keys():
    s += keyw + ", "
print s
    
with open("/home/cloudera/Desktop/text_mining/LDAkeyw.txt", 'a') as f:     
    f.write(s.encode("utf8"))
    f.close()

性侵, 當代, 藥物, 劇, 去年, 黃春明, 美食, 商機, 女網友, 外, ：, 優惠, 郭仕煌, 程度較高, 土銀, 許女, 發展, 系列, 華航, 所以, tw, 選區, 坪林, 案件, 楊女, to, 飯店, 頂新, 勸募, 生肖, 電子支付, 業績, 服務, 希望, 各地, 放, 第二艘, 千璽, 分鐘, CONVERSE, 建造, 設計, 聯盟, 警員, 平反, 表面, 以, 徐莉玲, 空難, BOOST, 博物館, 遊行, 除, 工程, 出國, 民進黨, 桂蘭, 罪, 館外, 北部, 聰, 蠟燭, 歲, 年, 席, 荒山亮, 勇士, 書店, 滿, 市議員, ％, 與, 經濟學, 個, 瘦肉精, 班上, 投資人, 肛門, ５, 星伊, 外套, 退休, 羽絨, 增資, 嘉義, 姐姐, 標靶藥物, Janet, 腫瘤, 九十九, 自己的, 卡牌, 是, 色, 低點, 台彩, 富邦, 可是, 資格賽, 則, 高苑工商, 訂位, 寇謐將, 內容, ◎, iRead, 適用, 雪蓮, 美國, 加強, 準備, 舞者, 也, 瘤, 我們, 手臂, 車款, 夏台鳳, 兩成, 老師, 二年, 高公局, 售價, 周美青, 無法, 更, 年度, Yahoo奇摩拍賣, 捐贈, 塞爾提克, 被, 入園, 三星, 未, 小火, 目前, adidas, 詠春, 粉, 警方, 上, 熟成, 惠玲, 判決, 喜特麗, 顯得, 資料夾, 攝氏, 花車, 緩刑, 聲援, 女方, 行情, 張程翔, 男友, 第, 送修, 照片, 圓明園, 交換, 金管會, 大眾, 湖人, 男子, 優秀, 創造, 巴士, 讓人, 百, 適量, 打發, 夜夜, 住院, 中國, 視網膜, 議會, 董事長, 電腦, 微軟, 油花, 新屋, 選舉, 變數, 以上, 流浪貓, SW, 柯, 手術, 說明, 該, 即可, 學生, 潘女, 網路, 老, 護理師, 化療, 油脂, ★, 基質, 哥哥, 罹患, 取得, 們, 起訴, 對話, nVIDIA, YEEZY, 推出, 期, 東北季風, 二億, 乳房, 女大生, 結婚, 眼霜, 續養, 陳, 萊克多巴胺, 台幣, 殷仔, 開幕, 威士忌, 人, 類別, 立院, 演出, 辭退, 三環三線, 兩款, 公斤, 克, 洪男, 才, 飼主, 跨年, 民眾, 球星, 沈國榮, 全年

In [114]:
#建立稀疏矩陣,並存檔
#每一行代表一篇文章的關鍵詞word count, 但大部分會是0次, 因為沒切出那個關鍵詞
#比如:
# 0 0 0 0 0 0 0 3 0 0 0 7 0 0 0 0 0 0 0 0 2
# 1 0 0 0 2 0 0 0 0 1 0 0 0 0 0 0 0 4 0 0 1

s = ""
for coll in wc_collect:
    for val in coll.values():
        s += str(val) + " "
    s += "\n"
    
with open("/home/cloudera/Desktop/text_mining/LDAvalues.txt", 'w') as f:     
    f.write(s.encode("utf8"))
    f.close()

In [135]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# 實作LDA

data = sc.textFile("file:/home/cloudera/Desktop/text_mining/LDAvalues.txt")
# 讀入txt檔, 每一行當作一個字串
#print data.collect()
#print 

parsedData = data.map(
    lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
#print parsedData.collect()
#print 

# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()
#print corpus.collect()
#print 

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=20) # 分成k個主題

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + \
      str(ldaModel.vocabSize()) + " words):")

topics = ldaModel.topicsMatrix()
for topic in range(10): # 印出10個主題的結果
    i = 0
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        if topics[word][topic]>10:               # 列出topic相關度大於10的
            print wc_collect[0].keys()[i],(" " + str(topics[word][topic]))
        i += 1
    print   

# Save and load model   # 存取模型
ldaModel.save(sc, "file:/home/cloudera/Desktop/text_mining/LDAmodel")
#ldaModel = LDAModel.load(sc, "file:/home/cloudera/Desktop/text_mining/LDAmodel")

Learned topics (as distributions over vocab of 1061 words):
Topic 0:
年  15.0428624293
高雄市  10.0304359577
投資  13.7648268435
池城  13.1935923069
材料  10.2112843456
膠囊  10.7821031207
天山  19.4180867873
偷拍  12.8008215248
他  43.9963365564

Topic 1:
：  20.6229522623
年  18.745338939
是  14.7628921132
第  10.2651496805
期  13.752825383
水腫  15.0488733562
壞帳  29.7693225252
肌膚  11.5754195864
交保  13.613257696
基金會  12.8840697994
分行  10.5462506162
營收  11.0283583805
室溫  10.9373710531
持續  23.4003046206
和大  10.9373710531

Topic 2:
手術  12.9583432631
達文西手術  15.8307879421
香港  22.4943387042
在  24.9524920811
銅鑼灣  10.2076589158
遊戲  16.8314466976
系統  13.1365643105
桂民海  14.7794231995

Topic 3:
◎  22.2159660219
第  18.6642595015
量級  10.9439410918
鐵板燒  18.5052961954
之  10.9314773327
桂民海  14.3444222632
Netflix  10.7982954891

Topic 4:
：  22.807347019
★  33.9257354017
人  12.9127188745
選票  10.7756756715
坪  14.9539899938
廖女  13.954012519
甄子丹  14.6660405763
魔女  21.3922583461
千個  12.9588232728
萬  28.7044536118
檢察官  11.7178272